In [198]:
import pulp
import numpy as np


###################################################################################
# DEFINE THE PROBLEM
###################################################################################

# Define the problem
prob = pulp.LpProblem("BasicLP",pulp.LpMinimize)

# Variables
x1 = pulp.LpVariable("x1",0,None)
x2 = pulp.LpVariable("x2",0,None)

# Objectives (defined as variables)
f1 = pulp.LpVariable("f1",None,None)
f2 = pulp.LpVariable("f2",None,None)

# Objective function (defined as constraints, to be minimized)
prob += f1 == 10*x1 + 1*x2
prob += f2 == 1*x1 + 10*x2

# Constraints
prob += 4*x1 + 1*x2 >= 8
prob += 1*x1 + 1*x2 >= 4
prob += 1*x1 + 8*x2 >= 8
prob += x1 + x2 <= 10

# List of objective functions (does not belong to prob object)
objective_function_names = ["f1","f2"] 
objective_functions = [f1,f2]

###################################################################################

class MOLP:
    def __init__(self,prob,objective_functions):
        self.prob = prob
        self.objective_functions = objective_functions
        self.num_objectives = len(objective_functions)
        self.num_variables = len(prob.variables())

        # Attributes to be filled during the optimization process (using the class methods)
        self.individual_optima = None # List of individual optima (solution object)
        self.payoff_table = None # Payoff table (num_objectives x num_objectives numpy array)
        self.ideal_point = None # Ideal point (numpy array)
        self.nadir_point = None # Nadir point (numpy array)

    def compute_individual_optima(self):
        self.individual_optima = []
        for f in self.objective_functions:
            self.prob.setObjective(f)
            self.prob.solve()
            self.individual_optima.append(Solution(self.prob))
        return self.individual_optima

    def compute_payoff_table(self):
        # Compute individual optima if not already computed
        if self.individual_optima is None:
            self.compute_individual_optima()
        # Compute payoff table
        self.payoff_table = np.zeros((self.num_objectives, self.num_objectives))
        for i in range(self.num_objectives):
            for j in range(self.num_objectives):
                self.payoff_table[i, j] = self.individual_optima[i].objective_values[j]
        return self.payoff_table

    def compute_ideal_point(self):
        # Compute payoff table if not already computed
        if self.payoff_table is None:
            self.compute_payoff_table()
        # Compute ideal point
        self.ideal_point = np.min(self.payoff_table,axis=0)
        return self.ideal_point
    
    def compute_nadir_point(self):
        # Compute payoff table if not already computed
        if self.payoff_table is None:
            self.compute_payoff_table()
        # Compute nadir point
        self.nadir_point = np.max(self.payoff_table,axis=0)
        return self.nadir_point

    def normalize_objective_values(self,objective_values):
        # Compute ideal and nadir points if not already computed
        if self.ideal_point is None:
            self.compute_ideal_point()
        if self.nadir_point is None:
            self.compute_nadir_point()
        # Normalize objective values
        return (objective_values - self.ideal_point) / (self.nadir_point - self.ideal_point)

class Solution():
    def __init__(self,prob=None):
        self.id = None # Unique identifier
        self.prob = prob
        if prob is not None:
            self.objective_values = np.array([v.varValue for v in self.prob.variables() if v.name in objective_function_names])
            self.objective_dict = {v.name: v.varValue for v in self.prob.variables() if v.name in objective_function_names}
            self.decision_values = np.array([v.varValue for v in prob.variables() if not v.name in objective_function_names])
            self.decision_dict = {v.name: v.varValue for v in prob.variables() if not v.name in objective_function_names}
            self.execution_time = prob.solutionTime
        else:
            self.objective_values = None
            self.decision_values = None
            self.execution_time = None

    # Set an arbitrary point of the decision space (given by a dictionary)
    def set_decision_dict(self,arbitrary_decision_dict):
        self.decision_dict = arbitrary_decision_dict
        self.decision_values = np.array([v for k,v in arbitrary_decision_dict.items])

    # Set an arbitrary point of the objective space (given by a dictionary)
    def set_objective_dict(self,arbitrary_objective_dict):
        self.objective_dict = arbitrary_objective_dict
        self.objective_values = np.array([v for k,v in arbitrary_objective_dict.items])


problem = MOLP(prob,objective_functions)
problem.compute_individual_optima()
problem.compute_payoff_table()
problem.compute_ideal_point()
problem.compute_nadir_point()

print('First sol')
print(problem.individual_optima[0].objective_values)
print(problem.individual_optima[0].decision_values)
print('Seoc sol')
print(problem.individual_optima[1].objective_values)
print(problem.individual_optima[1].decision_values)
print('Payoff table')
print(problem.payoff_table)
print('Ideal point')
print(problem.ideal_point)
print('Nadir point')
print(problem.nadir_point)

problem.individual_optima[0].objective_dict

First sol
[ 8. 80.]
[0. 8.]
Seoc sol
[80.  8.]
[8. 0.]
Payoff table
[[ 8. 80.]
 [80.  8.]]
Ideal point
[8. 8.]
Nadir point
[80. 80.]


{'f1': 8.0, 'f2': 80.0}

In [269]:
from copy import deepcopy, copy

def distribute_line_points(A, B, K):
    A, B = np.array(A), np.array(B)
    t = np.linspace(0, 1, K)
    return np.array([(1 - ti) * A + ti * B for ti in t])

def distribute_triangle_points(A, B, C, K):
    A, B, C = np.array(A), np.array(B), np.array(C)
    points = []
    for i in range(K):
        for j in range(K - i):
            a = i / (K - 1)
            b = j / (K - 1)
            c = 1 - a - b
            point = a * A + b * B + c * C
            points.append(point)
    return np.array(points)
    
class NormalBoundaryIntersection:
    def __init__(self, MOLPproblem, num_reference_points=10):
        self.MOLPproblem = MOLPproblem # MOLP object
        self.prob = MOLPproblem.prob # pulp.LpProblem object
        self.num_objectives = MOLPproblem.num_objectives
        self.num_variables = MOLPproblem.num_variables
        self.num_reference_points = num_reference_points

        # Attributes from the problem, compute if not already computed
        # ... individual optima
        if MOLPproblem.individual_optima is not None:
            self.individual_optima = MOLPproblem.individual_optima
        else:
            self.individual_optima = MOLPproblem.compute_individual_optima()
        # ... payoff table
        if MOLPproblem.payoff_table is not None:
            self.payoff_table = MOLPproblem.payoff_table
        else: 
            self.payoff_table = MOLPproblem.compute_payoff_table()
        # ... ideal point
        if MOLPproblem.ideal_point is not None:    
            self.ideal_point = MOLPproblem.ideal_point
        else:
            self.ideal_point = MOLPproblem.compute_ideal_point()
        # ... nadir point
        if MOLPproblem.nadir_point is not None:
            self.nadir_point = MOLPproblem.nadir_point
        else:
            self.nadir_point = MOLPproblem.compute_nadir_point()

        # Attributes to be filled during the NBI method (using the class methods)
        self.reference_points = None
        self.normal_vector = None # Normal vector to the hyperplane defined by the individual optima
        self.representative_pareto_front = None # List of solutions
        self.reference_point_to_solution = None # Dictionary that maps reference points to solutions

    def compute_reference_points(self):
        # For bi-objective problems, the reference points are distributed along the line connecting the individual optima
        if self.num_objectives == 2:
            A = self.individual_optima[0].objective_values
            B = self.individual_optima[1].objective_values
            # Both inplace and return:
            self.reference_points = distribute_line_points(A, B, self.num_reference_points)
            return self.reference_points
        
        # For tri-objective problems, the reference points are distributed in the triangle formed by the individual optima
        elif self.num_objectives == 3:
            A = self.individual_optima[0].objective_values
            B = self.individual_optima[1].objective_values
            C = self.individual_optima[2].objective_values
            # Both inplace and return:
            self.reference_points = distribute_triangle_points(A, B, C, self.num_reference_points)
            return self.reference_points
        
    def compute_normal_vector(self):
        # Compute the normal vector to the hyperplane defined by the individual optima
        # This vector should point to the half-space where the ideal point is located
        if self.num_objectives == 2:
            A = self.individual_optima[0].objective_values
            B = self.individual_optima[1].objective_values
            direction_vector = B - A
            normal_vector_1 = np.array([-direction_vector[1], direction_vector[0]]) 
            normal_vector_2 = -normal_vector_1
            # Determine which normal vector points to the half-space where the ideal point is located (TO BE CHECKED)
            if np.dot(normal_vector_1, self.ideal_point - A) > 0:
                self.normal_vector = normal_vector_1 / np.linalg.norm(normal_vector_1)
            else:
                self.normal_vector = normal_vector_2 / np.linalg.norm(normal_vector_1)
        return self.normal_vector
    
    def solve_NBI_subprolbem(self, reference_point):
        nbi_subproblem = deepcopy(self.prob)
        normal_vector = self.compute_normal_vector()
        # Set new variable t, objective function max t, and constraints reference_point + t * normal_vector == f
        t = pulp.LpVariable("t",0,None)
        #nbi_subproblem.setObjective(t)
        for i in range(self.num_objectives):
            nbi_subproblem += reference_point[i] + t * normal_vector[i] == self.MOLPproblem.objective_functions[i]
        nbi_subproblem.solve()
        #print('Status:', [v.varValue for v in nbi_subproblem.variables() if v.name in objective_function_names])
        # Return the solution
        nbi_solution = Solution(nbi_subproblem)
        #print('SubNBI: ', nbi_solution.objective_dict)

        return nbi_solution

    def compute_representative_pareto_front(self):
        # If not already computed, compute reference points
        if self.reference_points is None:
            self.compute_reference_points()

        # Compute the representative Pareto front
        self.representative_pareto_front = []
        self.reference_point_to_solution = {}

        # Solve NBI subproblems for each reference point
        for reference_point in self.reference_points:
            nbi_solution = self.solve_NBI_subprolbem(reference_point)
            # Add the solution to the representative Pareto front
            self.representative_pareto_front.append(nbi_solution)
            self.reference_point_to_solution[tuple(reference_point)] = nbi_solution
        return self.representative_pareto_front


problem = MOLP(prob,objective_functions)
NBI_example = NormalBoundaryIntersection(problem)

NBI_example.compute_reference_points()
print('Reference points')
print(NBI_example.reference_points)

NBI_example.compute_normal_vector()
print('Normal vector')
print(NBI_example.normal_vector)

NBI_example.compute_representative_pareto_front()
print('Representative pareto front')
print([solution.objective_values for solution in NBI_example.representative_pareto_front])
print('Solution time')
print([solution.execution_time for solution in NBI_example.representative_pareto_front])
    

Reference points
[[ 8. 80.]
 [16. 72.]
 [24. 64.]
 [32. 56.]
 [40. 48.]
 [48. 40.]
 [56. 32.]
 [64. 24.]
 [72. 16.]
 [80.  8.]]
Normal vector
[-0.70710678 -0.70710678]
Representative pareto front
[array([80.,  8.]), array([ 8., 80.]), array([10.133333, 66.133333]), array([12.266667, 52.266667]), array([14.4, 38.4]), array([18., 26.]), array([26., 18.]), array([34., 10.]), array([48.790123 ,  8.7901235]), array([64.395062 ,  8.3950617])]
Solution time
[0.015074491500854492, 0.014883279800415039, 0.014998435974121094, 0.013005495071411133, 0.012997627258300781, 0.026956796646118164, 0.018981218338012695, 0.028935670852661133, 0.01500082015991211, 0.018001556396484375]


In [125]:
problem = MOLP(prob,objective_functions)
NBI_example = NormalBoundaryIntersection(problem)
NBI_example.compute_reference_points(10)
print(NBI_example.reference_points)

[[8.0, 80.0],
 [16.0, 72.0],
 [24.0, 64.0],
 [32.0, 56.00000000000001],
 [40.0, 48.0],
 [48.0, 40.0],
 [55.99999999999999, 32.00000000000001],
 [63.99999999999999, 24.000000000000007],
 [72.0, 16.000000000000004],
 [80.0, 8.0]]

In [ ]:

###################################################################################
# CODE STRUCTURE
###################################################################################
class MultiObjectiveProblem:
    def __init__(self, name, pulp_lp_object, objective_functions):
        self.name = name
        self.objective_functions = objective_functions
        self.num_objectives = len(objective_functions)
        self.optimization_model = pulp_lp_object

    def evaluate_objectives(self, decision_vector):
        return [func(*decision_vector) for func in self.objective_functions]

    def solve_one_objective(self, objective):
        print(objective)
        self.optimization_model += objective
        self.optimization_model.solve()
        return {v.name: v.varValue for v in self.optimization_model.variables()}


myproblem = MultiObjectiveProblem('DummyLP', prob, objective_functions)
myproblem.solve_one_objective(f1(x1,x2))


In [30]:
import pulp

############################################################################
# Define the objective functions
def f1(x1, x2):
    return 10 * x1 + 1 * x2
def f2(x1, x2):
    return 1 * x1 + 10 * x2
############################################################################

class MultiObjectiveProblem:
    def __init__(self, name, objective_functions):
        ############################################################################
        # Define your problem here
        # Initialize the problem
        self.prob = pulp.LpProblem(name, pulp.LpMinimize)
        # Define the decision variables
        self.x1 = pulp.LpVariable("x1", 0, None)
        self.x2 = pulp.LpVariable("x2", 0, None)
        # Add the constraints
        self.prob += 4 * self.x1 + self.x2 >= 8
        self.prob += self.x1 + self.x2 >= 4
        self.prob += self.x1 + 8 * self.x2 >= 8
        self.prob += self.x1 + self.x2 <= 10
        ############################################################################

    def solve_one_objective(self, objective_function):
        # Define the objective function
        self.prob += objective_function(self.x1, self.x2)
        # Solve the problem
        self.prob.solve()
        # Return the solution
        return self.x1.varValue, self.x2.varValue



# Create an instance of MyProblem
myproblem = MultiObjectiveProblem()

# Solve the problem with f1 as the objective function
solution = myproblem.solve_one_objective(f1)
print(f"Solution for f1: x1 = {solution[0]}, x2 = {solution[1]}")

# Solve the problem with f2 as the objective function
solution = myproblem.solve_one_objective(f2)
print(f"Solution for f2: x1 = {solution[0]}, x2 = {solution[1]}")

Solution for f1: x1 = 0.0, x2 = 8.0
Solution for f2: x1 = 8.0, x2 = 0.0


c:\Users\Pol\AppData\Local\Programs\Python\Python312\Lib\site-packages\pulp\pulp.py:1650: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")
